In [ ]:
#pip install transformers torch
#pip install scikit-learn pandas nltk gensim pyLDAvis
#pip install spacy pandas scikit-learn
#pip install scispacy
#pip install medspacy
#pip install textacy
#pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_core_sci_md-0.5.1.tar.gz
import json
import requests
import pandas as pd
import nltk
from nltk.corpus import stopwords
import re
from transformers import pipeline, AutoTokenizer, AutoModelForTokenClassification
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import gensim
from gensim import corpora
import textacy.preprocessing.normalize as tnorm
import textacy.preprocessing.remove as trem
import spacy
import textacy.preprocessing as tp
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
import torch
import medspacy
nltk.download("stopwords")
nltk.download("punkt")
nltk.download('wordnet')
nlp = spacy.load("en_core_sci_md")


[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

This notebook explores various techniques to extract medical keywords from alredy prepared and cleaned transcript achieved before, including NER, Deepseek-r1, GPT-4 Turbo, LDA, QWEN2, and scispacy. All of techinques showed not very good result, however combining NER and gpt-4 we achieved good list which can be used on a platform to search this particular video and give some insights about content of surgery operation. Firstly executing NER and after GPT to filter and enhance the extracted results for more relevant medical insights.

In [ ]:
# Table of Contents
- [NER](#ner)
- [Deepseek-r1](#deepseek-r1)
- [GPT-4-turbo](#gpt-4-turbo)
- [LDA](#lda)
- [QWEN2](#qwen2)
- [scispacy](#scispacy)
- [Combine 2 approaches](#combine-2-approaches)
- [English text keywords extraction](#english-text)

In [ ]:
import openai

client = openai.OpenAI(api_key="ENTER OPENAI API KEY HERE")


In [ ]:
file_path = "./data/dorian2b_2text.txt" #Already prepared translated version of German text before
with open(file_path, "r", encoding="utf-8") as file:
    text = file.read()

## NER

### HUMADEX/english_medical_ner

In [ ]:
model_name = "HUMADEX/english_medical_ner"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits

predictions = torch.argmax(logits, dim=-1)

tokens = tokenizer.convert_ids_to_tokens(inputs.input_ids[0])
labels = [model.config.id2label[prediction.item()] for prediction in predictions[0]]

for token, label in zip(tokens, labels):
    print(f"{token}: {label}")


[CLS]: O
Test: E-TEST
Test: E-TEST
record: O
62: O
Fr: O
##e: O
##que: O
##ncy: O
Pre: O
##par: O
##ation: O
Pro: O
##ced: O
##ure: O
makes: O
sorry: O
finds: O
Ki: O
on: O
the: O
table: O
Video: O
works: O
,: O
unfortunately: O
let: O
': O
s: O
hope: O
the: O
sound: O
will: O
work: O
soon: O
too: O
.: O
Yeah: O
,: O
I: O
have: O
four: O
pieces: O
now: O
already: O
for: O
Christmas: O
.: O
Have: O
Kit: O
##a: O
app: O
family: O
app: O
Kit: O
##a: O
yeah: O
yeah: O
yeah: O
practical: O
you: O
know: O
look: O
in: O
.: O
W: O
##irs: O
##ing: O
,: O
Ko: O
##hl: O
,: O
apple: O
jam: O
,: O
c: O
##ro: O
##quet: O
##tes: O
,: O
very: O
impressive: O
,: O
delicious: O
Kit: O
##a: O
today: O
you: O
should: O
check: O
.: O
They: O
had: O
br: O
##occo: O
##li: O
,: O
uh: O
yeah: O
at: O
home: O
.: O
Mom: O
potato: O
soup: O
kindergarten: O
good: O
masses: O
home: O
potato: O
best: O
cook: O
appetite: O
eat: O
easy: O
yes: O
yes: O
oh: O
yes: O
funny: O
please: O
free: O
and: O
sorry: O
find: O
re

Each token was associated with a label. But unfortunately this result is not good, because 99.9% of tokens associated with  O, which means Outside (not part of a named entity) and only one word test is is assigned to E-TEST label , which means medical test.


### Bio_ClinicalBERT

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
model = AutoModelForTokenClassification.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)

with torch.no_grad():
    outputs = model(**inputs)

logits = outputs.logits

predictions = torch.argmax(logits, dim=2)

tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])

for token, prediction in zip(tokens, predictions[0].numpy()):
    if prediction != 0:
        print(f"Token: {token}, Entity: {model.config.id2label[prediction]}")


Some weights of BertForTokenClassification were not initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Token: test, Entity: LABEL_1
Token: test, Entity: LABEL_1
Token: record, Entity: LABEL_1
Token: ##i, Entity: LABEL_1
Token: table, Entity: LABEL_1
Token: video, Entity: LABEL_1
Token: unfortunately, Entity: LABEL_1
Token: hope, Entity: LABEL_1
Token: sound, Entity: LABEL_1
Token: will, Entity: LABEL_1
Token: soon, Entity: LABEL_1
Token: already, Entity: LABEL_1
Token: ##rist, Entity: LABEL_1
Token: have, Entity: LABEL_1
Token: family, Entity: LABEL_1
Token: app, Entity: LABEL_1
Token: look, Entity: LABEL_1
Token: in, Entity: LABEL_1
Token: k, Entity: LABEL_1
Token: ##oh, Entity: LABEL_1
Token: ##l, Entity: LABEL_1
Token: apple, Entity: LABEL_1
Token: c, Entity: LABEL_1
Token: ##ro, Entity: LABEL_1
Token: ., Entity: LABEL_1
Token: br, Entity: LABEL_1
Token: at, Entity: LABEL_1
Token: ., Entity: LABEL_1
Token: masses, Entity: LABEL_1
Token: home, Entity: LABEL_1
Token: easy, Entity: LABEL_1
Token: yes, Entity: LABEL_1
Token: yes, Entity: LABEL_1
Token: yes, Entity: LABEL_1
Token: and, En

In [ ]:
print(model.config.id2label)


{0: 'O', 1: 'B-PROBLEM', 2: 'I-PROBLEM', 3: 'E-PROBLEM', 4: 'S-PROBLEM', 5: 'B-TREATMENT', 6: 'I-TREATMENT', 7: 'E-TREATMENT', 8: 'S-TREATMENT', 9: 'B-TEST', 10: 'I-TEST', 11: 'E-TEST', 12: 'S-TEST'}


Very poor result for this model, it doesnt find any catogories except of B-problems, which means Beginning of a "PROBLEM" entity (e.g., a medical issue like a disease, symptom, or condition , but a lot of words which were labeled are not realted to medicine.

In [ ]:
model_name = "HUMADEX/english_medical_ner"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits

predictions = torch.argmax(logits, dim=-1)

tokens = tokenizer.convert_ids_to_tokens(inputs.input_ids[0])
labels = [model.config.id2label[prediction.item()] for prediction in predictions[0]]

extracted_keywords = []
current_entity = []

for token, label in zip(tokens, labels):
    if label != "O":
        if "B-" in label:
            if current_entity:
                extracted_keywords.append(" ".join(current_entity))
            current_entity = [token]
        elif "I-" in label:
            current_entity.append(token)
        elif "E-" in label:
            current_entity.append(token)
            extracted_keywords.append(" ".join(current_entity))
            current_entity = []
if current_entity:
    extracted_keywords.append(" ".join(current_entity))

print("Extracted Keywords:", extracted_keywords)


Extracted Keywords: ['Test', 'Test', 'gloves', 'gloves']


Very poor result for this model, nothing good labeled and identified

### Clinical-AI-Apollo/Medical-NER

In [ ]:
class MedicalNERExtractor:
    def __init__(self, model_name="Clinical-AI-Apollo/Medical-NER"):
        self.ner_pipeline = pipeline("token-classification", model=model_name, aggregation_strategy="simple", device=0)  # Use GPU


    def extract_keywords_in_batches(self, text, batch_size=3):
        if not isinstance(text, str):
            raise ValueError(f"Expected 'text' to be a string, but got {type(text)}")

        sentences = text.split(". ")
        extracted_terms = {}

        for i in range(0, len(sentences), batch_size):
            batch = ". ".join(sentences[i:i+batch_size])
            ner_results = self.ner_pipeline(batch)

            for entity in ner_results:
                term = entity["word"]
                category = entity["entity_group"]
                if category not in extracted_terms:
                    extracted_terms[category] = set()
                extracted_terms[category].add(term)

        return extracted_terms



    def display_matrix(self, extracted_terms):
        print("\nMedical Keywords Matrix:")
        print("=" * 50)
        for category, terms in extracted_terms.items():
            print(f"{category}: {', '.join(terms)}")
        print("=" * 50)

In [ ]:
ner_extractor = MedicalNERExtractor()

extracted_terms = ner_extractor.extract_keywords_in_batches(text)
ner_extractor.display_matrix(extracted_terms)

Device set to use cpu
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



Medical Keywords Matrix:
DETAILED_DESCRIPTION: org, on the side, venous, through, birthday, to, balloon a bit pushed, Middle bit of shit, Kita app family app Kita, heavy, knitting pattern, far down lower, ai training, umgestellt, contrast medium, straight, Kohl, wall, tilts down, sinking, phase, 62, above, a bit back, book, completely, high up, old guy, bit lower side, right, small, top, direction, ', behind the poster, gebogen, P, missing, second floor, s, came back back, crazy, down, cool, in, 180 seconds, balloon, bit more directly, twisted turns, branch, never quite right, more nice yes, bottom going down bottom stands, once or twice, back, bettel joyce Herr, piece, unclear, pp, start at the bottom, se, the left, retract, two, 123, t, push, bit, bend, beach, lett, Upper, in just corner above strong, Monday, under the lower, shift, front, za, ss, connection cable, mood, croquettes, Test Test, people license, r, Wirsing, -guided, upper, standing, z, r r, go back a, back back, bit la

We got a lot of useful words, but we also have a lot of garbage. Moreover, I do not call garbage certain categories that can be filtered out, but the fact that in the "useful" categories there is also an admixture of superfluous words that even relate to medicine or the text, but are not unique to the case.

## Deepseek-r1

In [ ]:
prompt_keywords = """
You are an AI with medical knowledge, and you are tasked with reading a text from a video of a medical operation. Please answer the following questions by identifying important and relevant keywords.

Act as though your input is a medical case.

Please respond only in plain text without using Markdown formatting, bullet points, headings, or links.

Examine the raw transcription of the medical operation video. While formulating your responses, never mention the transcript. Speak as if your information comes directly from the video recording of the operation.

Ensure that all details remain true to the original transcription without altering or simplifying the language. The language should remain true to the original transcript, preserving all medical terms, specific procedures, anatomy, medications, diagnostic tools, and any relevant keywords.

Provide a list of the following categories of keywords:

- Medical procedures: any surgical procedures, actions, or techniques performed during the operation.
- Medical tools/devices: surgical instruments, devices, or equipment used.
- Anatomical locations: body parts, organs, or anatomical locations mentioned.
- Medications: drugs, medications, or anesthesia mentioned.
- Symptoms or conditions: any relevant medical conditions or symptoms described.
- Diagnostic procedures: any diagnostic techniques or tests referenced.

Ensure you only return relevant keywords for each category, without any additional text or explanations.
"""



In [ ]:
OLLAMA_ENDPOINT ="http://127.0.0.1:11434/api/generate"

payload = {
    "model": "deepseek-r1:7b",
    "prompt": prompt_keywords2,
    "stream": False
}

response = requests.post(OLLAMA_ENDPOINT, json=payload)

if response.status_code == 200:
    result = response.json()
    print(result["response"])
else:
    print("Error:", response.status_code, response.text)


<think>
Okay, so I just got this really long message. It looks like someone was trying to process some kind of technical issue with a CAT scan or maybe imaging equipment? The user is talking about PUS (pulmonary venous sinus), slugs, balloons, and stuff related to medical imaging.

First, they mention "PUS 3123 missing completely." I'm not exactly sure what that refers to. Maybe it's a specific term in their field or a typo. Then there are mentions of anatomy, temperatures, isolation times, freezing, pumping, and even some emotional language like "frozen" and "desolation."

I think the user might be struggling with understanding an error message related to medical imaging equipment. They're probably frustrated because the system isn't working as expected. The context includes technical terms that are specific to their field, which suggests they have some background in medicine or imaging.

They also seem confused about what's going wrong and how to fix it. There's a lot of repetition a

The deepseek-r1 can extract keywords, but the output format, despite the instructions given, does not suit us. Moreover, the answer shows that the model has difficulties in medical terminology, you need to try gpt.

## GPT 4 turbo

In [ ]:
response = client.chat.completions.create(
    model="gpt-4-turbo",
    messages=[{"role": "system", "content": prompt_keywords}],
    temperature=0,
    max_tokens=4000
)

print(response.choices[0].message.content)


Medical procedures:
- Laparoscopic cholecystectomy
- Incision
- Dissection
- Cauterization
- Extraction

Medical tools/devices:
- Laparoscope
- Trocar
- Surgical scissors
- Cautery tool
- Suction device

Anatomical locations:
- Gallbladder
- Cystic duct
- Cystic artery
- Liver
- Abdomen

Medications:
- General anesthesia
- Antibiotics

Symptoms or conditions:
- Cholecystitis
- Gallstones

Diagnostic procedures:
- Ultrasound
- Blood tests


There is quite clear and concise information here, and without hallucinations, all these words were used during the operation, this can be seen in the transcript too. However, the number of words is quite concise, I will try to increase it using another prompt with additional points for searching or after other methods of keyword extraction. Also what is important here I used temperature = 0! This is very important rule which directly affects the variability and randomness of generated responses. A lower LLM temperature value (0) produces more deterministic and focused outputs, ideal for tasks requiring factual accuracy. We should never increase it and never let this parameter be setted by default.

In [ ]:
prompt_keywords2 = """
You're an AI with medical knowledge, and you're asked to read a text from a video and answer the questions that are asked to you.
Act like your input is a medical case. Please respond in plain text without using Markdown formatting or special characters for headings, bullet points, code blocks, or links.

Examine the raw transcription of the medical operation video. While formulating your responses, never mention the transcript. Speak as if your information comes directly from the video recording of the operation. Ensure that all details remain true to the original transcription without altering or simplifying the language. Preserve all medical terminology as it appears in the transcript, ensuring fidelity to the original wording.

Categories for Keyword Extraction:
- **Medical Procedures:** Any and all surgical or medical procedures performed during the operation. Include both major and minor techniques, steps, or phases of the operation.
- **Medical Tools/Devices:** List all tools, devices, instruments, and equipment used during the procedure.
- **Anatomical Locations:** Include all body parts, organs, anatomical structures, and regions of the body mentioned.
- **Medications:** Any drugs, anesthesia, or medical substances mentioned.
- **Symptoms or Conditions:** Any medical symptoms, conditions, or diagnoses discussed.
- **Diagnostic Procedures:** Any imaging techniques, tests, or diagnostics mentioned in the transcription.
- **Therapeutic Interventions:** Include actions taken to treat or manage a condition, even if it's a part of the procedure.

Now, please extract the keywords from the following medical operation transcription and format your output as requested.

Transcription:
""" + text + """
"""

In [ ]:
response = client.chat.completions.create(
    model="gpt-4-turbo",
    messages=[{"role": "system", "content": prompt_keywords2}],
    temperature=0,
    max_tokens=4000
)

print(response.choices[0].message.content)


**Medical Procedures:**
- Venous ultrasound-guided procedures
- Appendicitis closure

**Medical Tools/Devices:**
- Gloves
- Treadmills
- Catheter
- Needle
- Balloon
- Contrast medium

**Anatomical Locations:**
- Appendix
- Vein
- Urethra
- Pulmonary veins
- Lung diaphragm

**Medications:**
- Heparin
- XY Loka (local anesthetic)

**Symptoms or Conditions:**
- Appendicitis

**Diagnostic Procedures:**
- Venous ultrasound

**Therapeutic Interventions:**
- Local anesthesia
- Appendicitis closure


Now the number of keywords is more expanded and it suits us. You can try other ways.

Lets try topic modeling - technique for uncovering the central topics and their distributions across a set of documents. Once we get topic we can look into topic what the words most predictive of the topic.
LDA = latent dialectic allocation. It gives weights for key words in a topic.

## LDA

In [ ]:
def preprocess_text(text):
    text = re.sub(r"\W", " ", text)
    text = text.lower()
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stopwords.words("english")]
    return " ".join(tokens)

processed_text = preprocess_text(text)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Екатерина\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Екатерина\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
documents = nltk.sent_tokenize(text)  # get individual sentences.

vectorizer = CountVectorizer(min_df=1, max_df=0.7, stop_words="english")
X = vectorizer.fit_transform(documents)

# LDA
lda = LatentDirichletAllocation(n_components=10, random_state=42)  #number of topics
lda.fit(X)

feature_names = vectorizer.get_feature_names_out()

for topic_idx, topic in enumerate(lda.components_):
    print(f"Topic #{topic_idx + 1}:")
    print(", ".join([feature_names[i] for i in topic.argsort()[-10:]]))
    print("\n")


Topic #1:
compressed, forward, consciously, understand, gate, stop, lung, yes, way, julius


Topic #2:
make, said, maybe, looks, just, good, really, yeah, yes, bit


Topic #3:
okay, actually, better, lower, really, good, look, yeah, bit, yes


Topic #4:
wide, damaged, yesterday, isolation, seconds, degrees, day, home, bit, ll


Topic #5:
echt, genug, set, steht, gut, mal, immer, ct, bisschen, ja


Topic #6:
minus, isolation, missing, wrong, stands, capture, start, good, break, right


Topic #7:
let, sorry, slit, seconds, drop, 90, works, hope, temperature, test


Topic #8:
mr, units, 10, 000, pretty, today, okay, good, bit, yes


Topic #9:
isolation, nice, temperature, ah, ask, little, freese, just, yeah, yes


Topic #10:
better, small, okay, really, come, exactly, just, little, yeah, yes




We see a lot of frequent common words, so let's try to clear them using the library and repeat them. Perhaps the number of topics should also be reduced from 10 to 7-8, depending on the result obtained further.

In [ ]:
def clean_text_spacy(text):
    doc = nlp(text)

    # Remove stopwords (reference to clean techniques comparison notebook why we use spacy)
    #punctuation is specially not removed because LDA requres sentences (number of documents)
    cleaned_text = " ".join([
        token.text for token in doc
        if not token.is_stop
    ])

    return cleaned_text

In [ ]:
clean_spacy = clean_text_spacy(text)
clean_spacy

'Test Test record 62 Frequency Preparation Procedure makes sorry finds Ki table Video works , unfortunately let hope sound work soon . Yeah , pieces Christmas . Kita app family app Kita yeah yeah yeah practical know look . Wirsing , Kohl , apple jam , croquettes , impressive , delicious Kita today check . broccoli , uh yeah home . Mom potato soup kindergarten good masses home potato best cook appetite eat easy yes yes oh yes funny free sorry find great thank says need drive bit little driving actually exactly 25:35 yes corner zero gravity agree yes nice yes emphasize come learn blessed easy learn learn guide bag hands best hands like teach like learn yes sure said yes means hand okay yes yes yes ah yes right yes R R R r r wrong wrong yeah getting okay easy Yes fits eggs yeah old uh open yeah goes yeah far folded yeah okay bag uh huh okay yeah yeah yeah right exactly yes super exactly start come yeah fit corner strong ah got yeah yeah yeah comes yeah yeah exactly lets crazy man old guy 

In [ ]:
documents = nltk.sent_tokenize(clean_spacy)  # get individual sentences.

vectorizer = CountVectorizer(min_df=1, max_df=0.7, stop_words="english")
X = vectorizer.fit_transform(documents)

lda = LatentDirichletAllocation(n_components=10, random_state=42)
lda.fit(X)

feature_names = vectorizer.get_feature_names_out()

for topic_idx, topic in enumerate(lda.components_):
    print(f"Topic #{topic_idx + 1}:")
    print(", ".join([feature_names[i] for i in topic.argsort()[-10:]]))
    print("\n")


Topic #1:
means, damn, nice, need, kita, day, okay, break, write, yes


Topic #2:
42, instead, break, minus, drop, seconds, yes, degrees, temperature, 90


Topic #3:
okay, 3123, mal, good, bisschen, lower, look, bit, ja, yes


Topic #4:
freese, lung, deeper, isolation, minimal, pm, water, phone, little, bit


Topic #5:
okay, lower, time, yes, today, maybe, oh, good, yeah, bit


Topic #6:
new, start, oh, works, yeah, isolation, course, better, come, yes


Topic #7:
let, actually, right, okay, exactly, look, good, bit, yeah, yes


Topic #8:
isolated, hmm, pds, honestly, question, quick, attached, arm, upper, ask


Topic #9:
let, press, worse, counter, rock, test, pretty, better, little, yeah


Topic #10:
look, yeah, lies, right, small, nice, thing, actually, little, yes




There are words that relate to medicine, but a lot of things discussed in the transcript are conversations, so the topics contain them. They are useless for us and unnecessary unnecessary words abound. But playing with number of topics we can adjust words, it can be seen below. More topics -> more useful words appear (because more detailed and concetrated info in the chunks of the text, but some of topics are completely conversational, so it need to be filtered in the end)

In [ ]:
documents = nltk.sent_tokenize(clean_spacy)

vectorizer = CountVectorizer(min_df=1, max_df=0.7, stop_words="english")
X = vectorizer.fit_transform(documents)

lda = LatentDirichletAllocation(n_components=7, random_state=42)
lda.fit(X)

feature_names = vectorizer.get_feature_names_out()

for topic_idx, topic in enumerate(lda.components_):
    print(f"Topic #{topic_idx + 1}:")
    print(", ".join([feature_names[i] for i in topic.argsort()[-10:]]))
    print("\n")


Topic #1:
delicious, impressive, croquettes, wirsing, apple, kohl, kita, ask, temperature, break


Topic #2:
90, small, good, seconds, thing, degrees, minus, actually, little, yes


Topic #3:
immer, okay, good, mal, bisschen, lower, look, ja, bit, yes


Topic #4:
time, nice, lies, 3123, upper, try, think, slit, little, yes


Topic #5:
yes, today, minimal, maybe, okay, oh, day, good, bit, yeah


Topic #6:
know, freese, nice, second, come, ah, isolation, better, yeah, yes


Topic #7:
little, actually, right, okay, look, exactly, good, bit, yeah, yes




In [ ]:
documents = nltk.sent_tokenize(clean_spacy)  # get individual sentences.

vectorizer = CountVectorizer(min_df=1, max_df=0.7, stop_words="english")
X = vectorizer.fit_transform(documents)

lda = LatentDirichletAllocation(n_components=20, random_state=42)
lda.fit(X)

feature_names = vectorizer.get_feature_names_out()

for topic_idx, topic in enumerate(lda.components_):
    print(f"Topic #{topic_idx + 1}:")
    print(", ".join([feature_names[i] for i in topic.argsort()[-10:]]))
    print("\n")


Topic #1:
location, 43, follows, freese, middle, minimal, pm, 14, deeper, uhr


Topic #2:
presses, underneath, ahead, swings, daughter, yesterday, today, lot, mood, good


Topic #3:
ah, yeah, steht, gut, ct, immer, mal, bisschen, yes, ja


Topic #4:
mentioned, adequate, rp, stop, drop, seconds, 90, good, water, wrong


Topic #5:
pull, lower, good, okay, break, maybe, minus, degrees, bit, yeah


Topic #6:
settings, yesterday, stands, bit, isolation, time, capture, start, yes, slit


Topic #7:
pretty, okay, 123, little, bit, lower, good, better, yes, yeah


Topic #8:
connected, hatch, ahead, annoys, underneath, presses, need, little, difference, honestly


Topic #9:
need, okay, nice, works, hope, day, julius, test, write, yes


Topic #10:
degrees, makes, second, know, freese, ask, nice, yes, yeah, temperature


Topic #11:
sorry, valve, premiere, light, shine, exchanged, pressure, bent, ah, nicely


Topic #12:
kita, come, yeah, pull, good, okay, lower, look, yes, bit


Topic #13:
right, n

I will try to use bigram = 2-words phrases.This can help capture more meaningful bc of completed medical terms. Lets see

In [ ]:
documents = nltk.sent_tokenize(clean_spacy)

vectorizer = CountVectorizer(min_df=1, max_df=0.7, stop_words="english", ngram_range=(2, 2))
X = vectorizer.fit_transform(documents)

lda = LatentDirichletAllocation(n_components=15, random_state=42)
lda.fit(X)

feature_names = vectorizer.get_feature_names_out()
for topic_idx, topic in enumerate(lda.components_):
    print(f"Topic #{topic_idx + 1}:")
    print(", ".join([feature_names[i] for i in topic.argsort()[-10:]]))  # Top 10 bigrams per topic
    print("\n")


Topic #1:
makes white, yeah focused, knows bilingual, know probably, yeah nice, okay yeah, 34 degrees, 180 seconds, freese lsv, second freese


Topic #2:
sound work, sorry finds, finds ki, 62 frequency, record 62, makes sorry, frequency preparation, table video, works unfortunately, yeah yeah


Topic #3:
team nursing, break okay, pds attached, premiere went, hatch connected, pushed forward, lung pushed, forward compressed, stop twice, compressed stop


Topic #4:
nice actually, okay ostium, white okay, yeah pieces, pieces christmas, isolation missing, yes tries, little higher, tries press, missing isolation


Topic #5:
yes ella, little open, good difficult, yes little, 26 pm, look right, 10 000, 000 units, mr parin, yes yes


Topic #6:
delicious kita, minus 42, break instead, good temperature, today check, kita today, 42 degrees, temperature drop, 90 seconds, slit slit


Topic #7:
feeling bit, yes adjust, adjust feeling, seconds second, second freeze, time 12, isolation time, pusch bit,

## QWEN2

In [ ]:
OLLAMA_ENDPOINT ="http://127.0.0.1:11434/api/generate"

payload = {
    "model": "qwen2:7b",
    "prompt": prompt_keywords2,
    "stream": False
}

response = requests.post(OLLAMA_ENDPOINT, json=payload)

if response.status_code == 200:
    result = response.json()
    print(result["response"])
else:
    print("Error:", response.status_code, response.text)


It appears to be a transcript of a conversation regarding medical procedures, specifically discussing heart catheterization and angiography. The dialogue seems to involve doctors or medical staff as they discuss their experiences and observations while performing these procedures on patients.

Some key points include:

1. Discussion about successfully completing procedures related to the heart including angiography and using contrast medium for better visualization.
2. Mention of using a balloon during the procedure which indicates that it might have been used in percutaneous coronary intervention (PCI) or similar interventional cardiology procedures.
3. Remarks suggesting they may need to perform more procedures on Monday, possibly referring to scheduling or patient needs, as well as wishing Eike goodbye for what seems to be an end-of-week or end-of-month situation.

The tone is informal and collaborative, indicative of a medical team discussing their work in real-time or afterwards d

In [ ]:
prompt_keywords3 = """
You are an AI with expert medical knowledge. Your task is to extract and organize key medical terms from a given transcript of a medical procedure.

### **Instructions:**
1. Only return the extracted keywords in a structured format. Do **not** provide explanations, summaries, or any additional text.
2. Categorize the extracted terms strictly under these headings:
    - **Medical Procedures:** [List of all surgical or medical procedures]
    - **Medical Tools/Devices:** [List of tools, instruments, or medical devices]
    - **Anatomical Locations:** [List of all body parts, organs, or anatomical locations]
    - **Medications:** [List of all drugs, anesthetics, or medications]
    - **Symptoms or Conditions:** [List of symptoms, diseases, or medical conditions]
    - **Diagnostic Procedures:** [List of diagnostic tests, scans, or procedures]
    - **Therapeutic Interventions:** [List of treatments or interventions]

3. Format your response **exactly** as shown in the example below:

""" + clean_spacy + """
"""

In [ ]:
OLLAMA_ENDPOINT ="http://127.0.0.1:11434/api/generate"

payload = {
    "model": "qwen2:7b",
    "prompt": prompt_keywords3,
    "stream": False
}

response = requests.post(OLLAMA_ENDPOINT, json=payload)

if response.status_code == 200:
    result = response.json()
    print(result["response"])
else:
    print("Error:", response.status_code, response.text)


It seems like this text is discussing medical procedures and the progress of a patient. Here are some key points:

1. A medical team performed angiography on a patient named Ali to view blood vessels and diagnose any issues.

2. They used a balloon catheter during the procedure, which needed to be precisely positioned for effective treatment or diagnosis. 

3. The radiation equipment was deactivated once the necessary procedures were completed safely. This likely means they've finished imaging tasks and moved on to other treatments or evaluations.

4. There's discussion about insulation levels and temperatures—indicating that they may have used heating methods during procedures like angioplasty, where temperature management is crucial for certain medical tools (like balloons).

5. They're also considering a "Christmas" theme for their work schedule in the future, possibly referring to holiday themes for patient communications or team events.

6. A contrast medium was administered but t

We didn't get the keywords here, so qwen2 is not suitable for this task.

## scispacy

In [ ]:
def extract_all_entities(text, min_entity_length=3):
    doc = nlp(text)

    extracted_entities = []
    for ent in doc.ents:
        if len(ent.text) < min_entity_length:
            continue
        extracted_entities.append({
            "text": ent.text,
            "label": ent.label_
        })

    return extracted_entities



In [ ]:
entities = extract_all_entities(text)

for entity in entities:
    print(f"Entity: {entity['text']}, Label: {entity['label']}")

Entity: Test Test record, Label: ENTITY
Entity: Frequency, Label: ENTITY
Entity: sound, Label: ENTITY
Entity: pieces, Label: ENTITY
Entity: Christmas, Label: ENTITY
Entity: Kita app family app, Label: ENTITY
Entity: Kita yeah yeah yeah, Label: ENTITY
Entity: practical, Label: ENTITY
Entity: apple jam, Label: ENTITY
Entity: croquettes, Label: ENTITY
Entity: impressive, Label: ENTITY
Entity: delicious Kita today you, Label: ENTITY
Entity: broccoli, Label: ENTITY
Entity: home, Label: ENTITY
Entity: potato soup, Label: ENTITY
Entity: home potato, Label: ENTITY
Entity: cook appetite, Label: ENTITY
Entity: eat, Label: ENTITY
Entity: corner zero, Label: ENTITY
Entity: gravity, Label: ENTITY
Entity: bag, Label: ENTITY
Entity: hands, Label: ENTITY
Entity: hands, Label: ENTITY
Entity: teach, Label: ENTITY
Entity: okay yes yes yes ah yes right yes R R R, Label: ENTITY
Entity: okay, Label: ENTITY
Entity: eggs, Label: ENTITY
Entity: folded down yeah, Label: ENTITY
Entity: okay, Label: ENTITY
Entity

In [ ]:
# these are medical-related labels
medical_labels = ["CHEMICAL", "DISEASE", "MEDICAL_PROCEDURE", "DRUG", "PHARMACEUTICAL", "TREATMENT", "THERAPY"]

extracted_entities = extract_key_medical_terms(text)
filtered_entities = [entity for entity in extracted_entities if entity['label'] in medical_labels]

for entity in filtered_entities:
    print(f"Entity: {entity['text']}, Label: {entity['label']}")


Nothing was found in medicine-related labels from this text. I will try to use medspacy

In [ ]:
nlp = medspacy.load()

In [ ]:
doc = nlp(text)

entities = []
for ent in doc.ents:
    entities.append((ent.text, ent.label_))

for entity in entities:
    print(f"Entity: {entity[0]}, Label: {entity[1]}")


Unfortunately, even the medspacy could not extract the necessary entities from the text. I will combine 2 approaches together.

## Combine 2 approaches NER + gpt for filter and enhancing

We see above that this NER model gave us the most rich output among others, but the issue was that most of them will be useless when searching for videos on the platform or understanding the case, so we decided to combine two approaches: first, get a lot of abundant entities from the HEP model, then create a prompt that will transfer all the entities to the gpt model, and also transfer it itself transcript and ask to filter the entities or enhanse with new ones.

In [ ]:
# combine both
prompt = f"""
I have the following medical terms extracted from an NER model:
{extracted_terms}

Here is the full transcript of the medical operation:
{clean_spacy}

Your task is to:
1. Filter the extracted terms to focus on the most important medical information. Remove irrelevant terms (e.g., numbers, unrelated words).
2. Enhance or add any missing medical terms or details that might be present in the transcript but were not captured by the NER model.
3. Provide the output in the following format:
    - Medical Procedures: [list of medical procedures or actions]
    - Medical Tools/Devices: [list of tools, instruments]
    - Anatomical Locations: [body parts, organs]
    - Medications: [drugs, medications]
    - Symptoms or Conditions: [conditions, symptoms]
    - Diagnostic Procedures: [diagnostic tests, techniques]
    - Therapeutic Interventions: [treatment actions or steps]

Please ensure that your response is well-organized, clear, and precise.
"""



In [ ]:
response = client.chat.completions.create(
    model="gpt-4-turbo",
    messages=[{"role": "system", "content": prompt}],
    temperature=0,
    max_tokens=4000
)

print(response.choices[0].message.content)

Based on the transcript and the extracted terms, here is a refined and enhanced list of medical information categorized appropriately:

**Medical Procedures:**
- Ultrasound-guided procedures
- Venous procedures
- Anesthetization (local)
- Catheter insertion (multipolar catheter in CS and transseptal function)
- Appendicitis closure

**Medical Tools/Devices:**
- Catheter (multipolar catheter)
- Ultrasound device
- EKG (Electrocardiogram) machine
- X-ray machine

**Anatomical Locations:**
- Appendix
- Pulmonary veins
- CS (Coronary Sinus)
- Lower arm
- Vein
- Intervertebral space
- Lung diaphragm

**Medications:**
- Heparin (1000 units mentioned)
- XY Loka (used for local anesthesia)

**Symptoms or Conditions:**
- Appendicitis
- Pain (implied by the need for local anesthesia and the mention of massaging a painful area)

**Diagnostic Procedures:**
- EKG (Electrocardiogram)
- X-rays
- Ultrasound (used for guiding procedures)

**Therapeutic Interventions:**
- Anesthetization (using XY Loka)

This approach has shown the best result among the others. The Apollo NER model can identify many necessary words, but it is replete with unnecessary and useless words, and then by sending them to gpt turbo using prompt engineering, thanks to the instructions given, you can achieve a cleaned shortened list and enriched with phrases that can also be useful.

# English text keywords extraction

## Pasteur_TMVR

In [ ]:
file_path = "./data/clean_past.txt"
with open(file_path, "r", encoding="utf-8") as file:
    pasteur = file.read()

In [ ]:
pasteur

'welcome clinic pastor transcatheter mitral valve valve case   Uh discussed together   baseline clinical Characteristic   imaging uh reviewed uh case uh uh detailing step step   transcatheter mitral valve valve implantation   also highlight um advanced feature periprocedural imaging   benefit guide us uh case   Uh case guided transesophageal echography colleague   doctor patient   course   reason general anesthesia care doctor   Um think without waiting day   briefly review done far     um mentioned   patient general dys benefit uh transesophageal echography guidance   Um right groin   already positioned uh um sheath trans valve implant uh uh 29 millimeter   According sizing   reviewed   small art line monitor arterial pressure take blood samples monitor act   uh uh image inside surgical frame   Um decided protect brain cerebral protection device   namely Sentinel   think   start   maybe review baseline echo images   Lauren   OK   mitral valve   see   leaflets pretty thin   prolapse an

In [ ]:
ner_extractor = MedicalNERExtractor()

extracted_terms = ner_extractor.extract_keywords_in_batches(pasteur)
ner_extractor.display_matrix(extracted_terms)

Device set to use cpu
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



Medical Keywords Matrix:
SIGN_SYMPTOM: conduction, regurgitation, ular, ival
LAB_VALUE: millimeters mercury valva


Very concise result, lets enhance with gpt-4-turbo

In [ ]:
# combine both
prompt = f"""
I have the following medical terms extracted from an NER model:
{extracted_terms}

Here is the full transcript of the medical operation:
{pasteur}

Your task is to:
1. **Filter** the extracted terms to focus on the most important medical information. Remove irrelevant terms (e.g., numbers, unrelated words).
2. **Enhance** or **add** any missing medical terms or details that might be present in the transcript but were not captured by the NER model.
3. Provide the output in the following format:
    - Medical Procedures: [list of medical procedures or actions]
    - Medical Tools/Devices: [list of tools, instruments]
    - Anatomical Locations: [body parts, organs]
    - Medications: [drugs, medications]
    - Symptoms or Conditions: [conditions, symptoms]
    - Diagnostic Procedures: [diagnostic tests, techniques]
    - Therapeutic Interventions: [treatment actions or steps]

Please ensure that your response is well-organized, clear, and precise.
"""



In [ ]:
response = client.chat.completions.create(
    model="gpt-4-turbo",
    messages=[{"role": "system", "content": prompt}],
    temperature=0,
    max_tokens=4000
)

print(response.choices[0].message.content)

Based on the transcript and the initial extraction from the NER model, here is an enhanced and organized summary of the medical terms and details relevant to the transcatheter mitral valve implantation procedure described:

**Medical Procedures:**
- Transcatheter mitral valve implantation
- Transeptal puncture
- Periprocedural imaging
- Baseline echo imaging
- Fluoroscopy
- Deployment of valve
- Pre-dilation of septum
- Post-dilation of valve

**Medical Tools/Devices:**
- Transesophageal echography (TEE)
- Sentinel cerebral protection device
- Sheath
- Balloon (14 millimeter for dilation)
- SAPIEN 3 valve (29 millimeter)
- Deflectable catheter
- Steerable capture
- Fluoroscopy equipment
- Pacing system

**Anatomical Locations:**
- Mitral valve
- Left atrium
- Left ventricle
- Supra Vena cava
- Inferior vena cava
- Inter atrial septum
- Ventricular part of prosthesis

**Medications:**
- General anesthesia
- Heparin (5000 units for anticoagulation)

**Symptoms or Conditions:**
- Mitral r

And now we are providing what our team extracted manually reading the text (without diploma in medicine):
* transcatheter mitral valve in valve case
* clinical Characteristic
* Color Doppler imaging
* transesophageal echography
* general anesthesia
* arterial pressure
* blood samples
* transeptal puncture
* supra Vena
* images of fusion
* X ray
* Radiopaque
* Aneurysm
* Fluoroscopy
* Atrium
* Heparin
* Septum
* stent frame
* perivalvular
* regurgitation
* hemodynamic


## AVAM_Mitraclip

This file is too large to process with NER so we will not combine both parts prvided by mednibox and will treat them separately. We will start from part 1.

In [ ]:
file_path = "./data/clean_avam1.txt"
with open(file_path, "r", encoding="utf-8") as file:
    avam = file.read()

In [ ]:
ner_extractor = MedicalNERExtractor()

extracted_terms = ner_extractor.extract_keywords_in_batches(avam, batch_size=2)
ner_extractor.display_matrix(extracted_terms)

Device set to use cpu
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



Medical Keywords Matrix:


Here NER did not find nothing, lets prompt got-4-turbo

In [ ]:
prompt = f"""
I have the following medical terms extracted from an NER model:
{extracted_terms}

Here is the full transcript of the medical operation:
{avam}

Your task is to:
1. **Filter** the extracted terms to focus on the most important medical information. Remove irrelevant terms (e.g., numbers, unrelated words).
2. **Enhance** or **add** any missing medical terms or details that might be present in the transcript but were not captured by the NER model.
3. Provide the output in the following format:
    - Medical Procedures: [list of medical procedures or actions]
    - Medical Tools/Devices: [list of tools, instruments]
    - Anatomical Locations: [body parts, organs]
    - Medications: [drugs, medications]
    - Symptoms or Conditions: [conditions, symptoms]
    - Diagnostic Procedures: [diagnostic tests, techniques]
    - Therapeutic Interventions: [treatment actions or steps]

Please ensure that your response is well-organized, clear, and precise.
"""



In [ ]:
response = client.chat.completions.create(
    model="gpt-4-turbo",
    messages=[{"role": "system", "content": prompt}],
    temperature=0,
    max_tokens=4000
)

print(response.choices[0].message.content)

Based on the transcript provided and the task requirements, here is the enhanced and organized list of medical terms and details:

**Medical Procedures:**
- Mitral valve repair
- Transcatheter mitral valve replacement (TMVR)
- Predilatation
- Deployment of valve
- Echocardiography
- Heart catheterization
- Surgical prosthesis implantation
- Microsurgery

**Medical Tools/Devices:**
- MitraClip
- Tendyne valve system
- Balloon catheter
- Echocardiography machine
- Surgical sutures
- Valve deployment system
- AMX trans catheter device removal system
- Cord cutters

**Anatomical Locations:**
- Mitral valve
- Left ventricular outflow tract (LVOT)
- Anterior leaflet of the mitral valve
- Left atrium
- Left ventricle
- Aortic root
- Pulmonary vein
- Coronary artery

**Medications:**
- Beta blockers
- Diuretics

**Symptoms or Conditions:**
- Left ventricular outflow tract obstruction (LVOT obstruction)
- Mitral regurgitation
- Mitral stenosis
- Heart failure
- Hemodynamic instability
- Arrhyth

Result achieved. Now lets see part 2 of this operation

In [ ]:
file_path = "./data/clean_avam2.txt"
with open(file_path, "r", encoding="utf-8") as file:
    avam2 = file.read()

In [ ]:
quarter = len(avam2) // 4
part_1 = avam2[:quarter]
part_2 = avam2[quarter:2*quarter]
part_3 = avam2[2*quarter:3*quarter]
part_4 = avam2[3*quarter:]

ner_extractor = MedicalNERExtractor()
extracted_terms_part1 = ner_extractor.extract_keywords_in_batches(part_1, batch_size=2)
extracted_terms_part2 = ner_extractor.extract_keywords_in_batches(part_2, batch_size=2)
extracted_terms_part3 = ner_extractor.extract_keywords_in_batches(part_3, batch_size=2)
extracted_terms_part4 = ner_extractor.extract_keywords_in_batches(part_4, batch_size=2)

Device set to use cpu
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
all_extracted_terms = {**extracted_terms_part1, **extracted_terms_part2, **extracted_terms_part3, **extracted_terms_part4}

ner_extractor.display_matrix(all_extracted_terms)


Medical Keywords Matrix:
DURATION: half hour, 30 minutes
DETAILED_DESCRIPTION: half, trophic posterior valve, are, trophic
BIOLOGICAL_STRUCTURE: peri, incision, olar, nipple half
LAB_VALUE: left, right, 12 6
DISEASE_DISORDER: bowel disease, bowel
SIGN_SYMPTOM: regurgitation, ventricular
HISTORY: imaging, surgeons


In [ ]:
prompt = f"""
I have the following medical terms extracted from an NER model:
{all_extracted_terms}

Here is the full transcript of the medical operation:
{avam2}

Your task is to:
1. **Filter** the extracted terms to focus on the most important medical information. Remove irrelevant terms (e.g., numbers, unrelated words).
2. **Enhance** or **add** any missing medical terms or details that might be present in the transcript but were not captured by the NER model.
3. Provide the output in the following format:
    - Medical Procedures: [list of medical procedures or actions]
    - Medical Tools/Devices: [list of tools, instruments]
    - Anatomical Locations: [body parts, organs]
    - Medications: [drugs, medications]
    - Symptoms or Conditions: [conditions, symptoms]
    - Diagnostic Procedures: [diagnostic tests, techniques]
    - Therapeutic Interventions: [treatment actions or steps]

Please ensure that your response is well-organized, clear, and precise.
"""



In [ ]:
response = client.chat.completions.create(
    model="gpt-4-turbo",
    messages=[{"role": "system", "content": prompt}],
    temperature=0,
    max_tokens=4000
)

print(response.choices[0].message.content)

Based on the transcript and the extracted terms from the NER model, here is an enhanced and organized list of medical information relevant to the discussed medical operation and procedures:

**Medical Procedures:**
- Mitral valve repair
- Tricuspid repair
- Pacemaker implantation
- Percutaneous treatment
- Surgical treatment
- Transseptal puncture
- Minimally invasive surgery
- Beating heart surgery
- Maze procedures
- Coronary angiogram
- MitraClip implantation
- Edge-to-edge repair
- Percutaneous repair
- Surgical case recording
- Valve assessment
- Sliding technique for valve repair
- Resection

**Medical Tools/Devices:**
- Pascal implant
- MitraClip
- Surgical instruments
- Imaging devices
- Echo analysis tools
- Guide wires (e.g., Safari wire)
- Clipping devices
- CT scan equipment
- MRI equipment
- Ultrasound machines

**Anatomical Locations:**
- Mitral valve
- Tricuspid valve
- Left atrium
- Right atrium
- Left ventricle
- Right ventricle
- Aortic valve
- Septum
- Left atrial ap

So if we combine both parts keywords generated by LLM we will get:

**Medical Procedures:**
* Mitral valve repair
* Transcatheter mitral valve replacement (TMVR)
* Predilatation
* Deployment of valve
* Echocardiography
* Heart catheterization
* Surgical prosthesis implantation
* Microsurgery
* Mitral valve repair
* Tricuspid repair
* Pacemaker implantation
* Percutaneous treatment
* Surgical treatment
* Transseptal puncture
* Minimally invasive surgery
* Beating heart surgery
* Maze procedures
* Coronary angiogram
* MitraClip implantation
* Edge-to-edge repair
* Percutaneous repair
* Surgical case recording
* Valve assessment
* Sliding technique for valve repair
* Resection

**Medical Tools/Devices:**
* MitraClip
* Tendyne valve system
* Balloon catheter
* Echocardiography machine
* Surgical sutures
* Valve deployment system
* AMX trans catheter device removal system
* Cord cutters
* Pascal implant
* MitraClip
* Surgical instruments
* Imaging devices
* Echo analysis tools
* Guide wires (e.g., Safari wire)
* Clipping devices
* CT scan equipment
* MRI equipment
* Ultrasound machines

**Anatomical Locations:**
* Mitral valve
* Left ventricular outflow tract (LVOT)
* Anterior leaflet of the mitral valve
* Left atrium
* Left ventricle
* Aortic root
* Pulmonary vein
* Coronary artery
* Mitral valve
* Tricuspid valve
* Left atrium
* Right atrium
* Left ventricle
* Right ventricle
* Aortic valve
* Septum
* Left atrial appendage

**Medications:**
* Beta blockers
* Diuretics
* Mra
* SGLT2 inhibitors

**Symptoms or Conditions:**
* Left ventricular outflow tract obstruction (LVOT obstruction)
* Mitral regurgitation
* Mitral stenosis
* Heart failure
* Hemodynamic instability
* Arrhythmias
* Mitral regurgitation
* Tricuspid regurgitation
* Heart failure
* Sinus node dysfunction
* Ventricular dysfunction
* Atrial fibrillation
* Regurgitation
* Dilated RVs (right ventricles)

**Diagnostic Procedures:**
* Echocardiography (transesophageal and transthoracic)
* CT scan
* Nuclear blood volume analysis
* Blood work
* Echocardiography (Echo)
* Coronary angiography
* Imaging studies
* 3D imaging
* Transthoracic echocardiogram (TTE)
* Transesophageal echocardiogram (TEE)

**Therapeutic Interventions:**
* Valve repair or replacement
* Use of beta blockers post*procedure
* Administration of diuretics
* Surgical removal of anterior leaflet
* Deployment of transcatheter valve
* Predilatation with a balloon to prepare the valve site
* Medical therapy optimization
* Surgical interventions
* Percutaneous interventions
* Device implantation
* Valve repair techniques
* Use of pharmacological treatments

And manually without any tools we extracted:

*	LDLT obstruction
*	Genetic
*	bone replacement
*	surgical prosthesis
*	beta blockers
*	right heart cath
*	microsurgery
*	pericardium
*	hemodynamic
*	aortic root
*	cardiac cycle
*	LV cavity
*	Surgical
*	Therapie
*	Regurgitation
*	Anatomy
*	Embolization
*	Ventricular
*	Catheter
*	mitral tricuspid
*	arrhythmias
*	MitraClip
*	Hypotensive
*	degenerative disease
*	progressive mitral stenosis
*	anesthesiologist
*	echocardiography
*	carotid endarectomy
*	atrium
*	transfemoral
*	transseptal
*	anatomical
*	predilatation
*	anus
*	pulmonary vein
*	AP diameter
*	Cardiologist
*	Invasive
*	sinus node
*	interventional therapies
*	Saline
*	Symptomatic
*	Trichopi
*	aortic bioprosthesis
*	angiogram
*	echo images
*	anesthesia
*	interventionalist
*	prolapse
*	procedure
*	ventricle
*	tissue
*	percutaneous technique
*	annulus
*	left atrial appendage
*	atrial fibrillation
*	diastole
*	systole
*	echocardiography
*	radiofrequency
*	mitral regurgitation
*	dense chordae
*	pharmacological treatment
*	primary care
*	secondary care
*	tertiary care
*	serial echo
*	heart failure
*	clinical inertia
*	transesophageal echocardiography
*	redo surgey
*	nitinal
*	stainless



We believe that LLM coped no worse than a human, but we need the opinion of real cardiologists. And its much more faster to obtain result than reading text by humans. (but we need human-in-the-loop at least sometimes to validate)